# Phase 3: Multi-Agent System
## Agentic AI for Wikipedia Article Generation

**Objective**: Build a multi-agent system that collaborates to generate high-quality Wikipedia-style articles.

**Agents we'll implement:**
1. **Research Agent** - Gathers information using GraphRAG
2. **Planning Agent** - Creates article outline and structure
3. **Writing Agent** - Generates coherent content sections
4. **Fact-Verification Agent** - Validates claims (simplified version)
5. **Orchestrator** - Coordinates the workflow

**What we'll do in this notebook:**
1. Load the GraphRAG engine from Phase 2
2. Set up a simple agent framework (using classes)
3. Implement each specialized agent
4. Create an orchestration workflow
5. Generate a complete Wikipedia article
6. Evaluate the output

**Note**: This is a simplified version without LLM API calls. We'll use templates and the GraphRAG system to demonstrate the architecture.

---

## Step 1: Import Libraries

Import required libraries for the agent system.

In [ ]:
# Core libraries
import os
import json
import pickle
from pathlib import Path
from typing import List, Dict, Any, Optional
from dataclasses import dataclass, field
from collections import defaultdict
from datetime import datetime

# Data processing
import pandas as pd
import numpy as np

# Graph and vector search
import networkx as nx
import faiss
from sentence_transformers import SentenceTransformer

# NLP
import spacy

# Progress tracking
from tqdm.auto import tqdm

print("✓ All libraries imported successfully!")

## Step 2: Load GraphRAG Engine

Load all components from Phase 2.

In [ ]:
# Define paths
PROJECT_ROOT = Path(r"d:\Projects\agent-wiki-graphrag")
DATA_DIR = PROJECT_ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"
EMBEDDINGS_DIR = DATA_DIR / "embeddings"
KG_DIR = DATA_DIR / "knowledge_graph"
OUTPUTS_DIR = PROJECT_ROOT / "outputs" / "articles"
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)

print("Loading GraphRAG engine components...")

# Load articles
with open(RAW_DIR / "wikipedia_articles.json", 'r', encoding='utf-8') as f:
    articles = json.load(f)
print(f"✓ Loaded {len(articles)} articles")

# Load knowledge graph
with open(KG_DIR / "article_graph.pkl", 'rb') as f:
    G = pickle.load(f)
print(f"✓ Loaded knowledge graph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

# Load FAISS index
faiss_index = faiss.read_index(str(EMBEDDINGS_DIR / "faiss_index.bin"))
with open(EMBEDDINGS_DIR / "index_titles.json", 'r', encoding='utf-8') as f:
    index_titles = json.load(f)
print(f"✓ Loaded FAISS index: {faiss_index.ntotal} vectors")

# Load embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print(f"✓ Loaded embedding model")

# Load entities
with open(PROCESSED_DIR / "entities.json", 'r', encoding='utf-8') as f:
    entities = json.load(f)
print(f"✓ Loaded entities for {len(entities)} articles")

print(f"\n{'='*60}")
print("GraphRAG Engine Ready!")
print(f"{'='*60}")

## Step 3: Rebuild Retriever Classes

Reconstruct the retriever classes from Phase 2.

In [ ]:
class VectorRetriever:
    """Vector-based retrieval"""
    def __init__(self, index, titles, encoder):
        self.index = index
        self.titles = titles
        self.encoder = encoder
    
    def search(self, query, top_k=10):
        query_vec = self.encoder.encode([query]).astype(np.float32)
        query_vec = np.ascontiguousarray(query_vec)
        faiss.normalize_L2(query_vec)
        scores, indices = self.index.search(query_vec, top_k)
        return [(self.titles[idx], float(scores[0][i])) 
                for i, idx in enumerate(indices[0]) if idx < len(self.titles)]

class GraphRetriever:
    """Graph-based retrieval"""
    def __init__(self, graph):
        self.graph = graph
    
    def multi_hop_retrieval(self, start_nodes, max_hops=2, max_results=50):
        if isinstance(start_nodes, str):
            start_nodes = [start_nodes]
        
        visited = set()
        results = []
        queue = [(node, 0) for node in start_nodes if node in self.graph]
        
        while queue and len(results) < max_results:
            current_node, current_hop = queue.pop(0)
            if current_node in visited:
                continue
            visited.add(current_node)
            results.append((current_node, current_hop))
            
            if current_hop < max_hops:
                neighbors = list(self.graph.successors(current_node))[:10]
                for neighbor in neighbors:
                    if neighbor not in visited:
                        queue.append((neighbor, current_hop + 1))
        return results

class HybridRetriever:
    """Hybrid retrieval"""
    def __init__(self, vector_retriever, graph_retriever, articles):
        self.vector = vector_retriever
        self.graph = graph_retriever
        self.articles = articles
    
    def retrieve(self, query, top_k=10, alpha=0.5):
        results = {}
        
        # Vector search
        vector_results = self.vector.search(query, top_k=top_k*2)
        for title, score in vector_results:
            results[title] = {'vector_score': score, 'graph_score': 0}
        
        # Graph search
        query_terms = query.lower().split()
        matching_articles = [t for t in self.articles.keys() 
                           if any(term in t.lower() for term in query_terms)]
        
        if matching_articles:
            graph_results = self.graph.multi_hop_retrieval(matching_articles[:3], 
                                                          max_hops=2, max_results=top_k*2)
            for title, hop in graph_results:
                if title not in results:
                    results[title] = {'vector_score': 0, 'graph_score': 0}
                results[title]['graph_score'] = 1.0 / (2 ** hop)
        
        # Normalize and rank
        max_v = max((r['vector_score'] for r in results.values()), default=1.0)
        max_g = max((r['graph_score'] for r in results.values()), default=1.0)
        
        ranked = []
        for title, scores in results.items():
            v_norm = scores['vector_score'] / max_v if max_v > 0 else 0
            g_norm = scores['graph_score'] / max_g if max_g > 0 else 0
            final_score = alpha * v_norm + (1 - alpha) * g_norm
            ranked.append((title, final_score))
        
        ranked.sort(key=lambda x: x[1], reverse=True)
        return ranked[:top_k]

# Initialize retrievers
vector_retriever = VectorRetriever(faiss_index, index_titles, embedding_model)
graph_retriever = GraphRetriever(G)
hybrid_retriever = HybridRetriever(vector_retriever, graph_retriever, articles)

print("✓ Retrievers initialized and ready!")

## Step 4: Define Agent State & Message Types

Create data structures for agent communication.

In [ ]:
@dataclass
class AgentMessage:
    """Message passed between agents"""
    agent: str
    content: Any
    timestamp: str = field(default_factory=lambda: datetime.now().isoformat())
    metadata: Dict = field(default_factory=dict)

@dataclass
class ArticleState:
    """State of the article generation process"""
    topic: str
    research_data: Dict = field(default_factory=dict)
    outline: List[str] = field(default_factory=list)
    sections: Dict[str, str] = field(default_factory=dict)
    citations: List[Dict] = field(default_factory=list)
    verification_results: Dict = field(default_factory=dict)
    final_article: str = ""
    messages: List[AgentMessage] = field(default_factory=list)
    
    def add_message(self, agent: str, content: Any, **metadata):
        """Add a message to the conversation"""
        msg = AgentMessage(agent=agent, content=content, metadata=metadata)
        self.messages.append(msg)
        return msg

print("✓ Agent state structures defined")

## Step 5: Implement Research Agent

Agent that gathers information using GraphRAG.

In [ ]:
class ResearchAgent:
    """Agent responsible for gathering information"""
    
    def __init__(self, retriever, articles):
        self.retriever = retriever
        self.articles = articles
        self.name = "Research Agent"
    
    def research(self, state: ArticleState, top_k=10) -> ArticleState:
        """Gather information about the topic"""
        print(f"\n{self.name}: Researching topic '{state.topic}'...")
        
        # Retrieve relevant articles
        results = self.retriever.retrieve(state.topic, top_k=top_k)
        
        # Gather context
        research_data = {
            'relevant_articles': [],
            'key_entities': defaultdict(int),
            'categories': defaultdict(int),
            'total_sources': len(results)
        }
        
        for title, score in results:
            if title in self.articles:
                article = self.articles[title]
                
                # Store article info
                article_info = {
                    'title': title,
                    'score': score,
                    'summary': article.get('summary_clean', '')[:500],
                    'url': article.get('url', ''),
                    'categories': article.get('categories', [])[:3]
                }
                research_data['relevant_articles'].append(article_info)
                
                # Aggregate entities
                if title in entities:
                    for ent in entities[title]:
                        research_data['key_entities'][ent['text']] += 1
                
                # Aggregate categories
                for cat in article.get('categories', []):
                    research_data['categories'][cat] += 1
        
        # Get top entities and categories
        research_data['key_entities'] = dict(
            sorted(research_data['key_entities'].items(), 
                   key=lambda x: x[1], reverse=True)[:20]
        )
        research_data['categories'] = dict(
            sorted(research_data['categories'].items(), 
                   key=lambda x: x[1], reverse=True)[:10]
        )
        
        state.research_data = research_data
        state.add_message(self.name, 
                         f"Gathered {len(results)} relevant sources",
                         sources=len(results))
        
        print(f"  ✓ Found {len(results)} relevant articles")
        print(f"  ✓ Identified {len(research_data['key_entities'])} key entities")
        print(f"  ✓ Top categories: {list(research_data['categories'].keys())[:3]}")
        
        return state

# Initialize research agent
research_agent = ResearchAgent(hybrid_retriever, articles)
print("✓ Research Agent initialized")

## Step 6: Implement Planning Agent

Agent that creates article structure and outline.

In [ ]:
class PlanningAgent:
    """Agent responsible for creating article outline"""
    
    def __init__(self):
        self.name = "Planning Agent"
    
    def plan(self, state: ArticleState) -> ArticleState:
        """Create article outline based on research"""
        print(f"\n{self.name}: Creating article outline...")
        
        research = state.research_data
        topic = state.topic
        
        # Basic Wikipedia article structure
        outline = [
            "Introduction",
            "Overview",
            "History",
            "Key Concepts",
            "Applications",
            "Challenges and Limitations",
            "Future Directions",
            "See Also",
            "References"
        ]
        
        # Customize based on topic
        if any(term in topic.lower() for term in ['machine learning', 'ai', 'algorithm']):
            outline = [
                "Introduction",
                "Definition and Overview",
                "History and Development",
                "How It Works",
                "Types and Variants",
                "Applications",
                "Advantages and Disadvantages",
                "Current Research",
                "See Also",
                "References"
            ]
        elif any(term in topic.lower() for term in ['quantum', 'physics']):
            outline = [
                "Introduction",
                "Overview",
                "Theoretical Background",
                "Key Principles",
                "Experimental Evidence",
                "Applications",
                "Interpretations",
                "Current Research",
                "See Also",
                "References"
            ]
        
        state.outline = outline
        state.add_message(self.name, 
                         f"Created outline with {len(outline)} sections",
                         sections=len(outline))
        
        print(f"  ✓ Created outline with {len(outline)} sections:")
        for i, section in enumerate(outline[:5], 1):
            print(f"    {i}. {section}")
        if len(outline) > 5:
            print(f"    ... and {len(outline) - 5} more")
        
        return state

# Initialize planning agent
planning_agent = PlanningAgent()
print("✓ Planning Agent initialized")

## Step 7: Implement Writing Agent

Agent that generates content for each section.

In [ ]:
class WritingAgent:
    """Agent responsible for writing article content"""
    
    def __init__(self, articles):
        self.articles = articles
        self.name = "Writing Agent"
    
    def write(self, state: ArticleState) -> ArticleState:
        """Write content for each section"""
        print(f"\n{self.name}: Writing article sections...")
        
        topic = state.topic
        research = state.research_data
        outline = state.outline
        
        sections = {}
        
        for section_title in outline:
            if section_title in ["References", "See Also"]:
                # Skip these, handle separately
                continue
            
            print(f"  Writing: {section_title}...")
            
            # Generate section content based on research
            content = self._generate_section(section_title, topic, research)
            sections[section_title] = content
        
        state.sections = sections
        state.add_message(self.name, 
                         f"Wrote {len(sections)} sections",
                         sections=len(sections))
        
        print(f"  ✓ Completed {len(sections)} sections")
        
        return state
    
    def _generate_section(self, section_title, topic, research):
        """Generate content for a specific section"""
        
        # Introduction
        if section_title == "Introduction":
            articles_list = research['relevant_articles'][:3]
            entities = list(research['key_entities'].keys())[:5]
            
            content = f"**{topic}** is a significant topic in modern research and application. "
            content += f"It is closely related to concepts such as {', '.join(entities[:3])}. "
            content += f"This article provides a comprehensive overview of {topic}, "
            content += f"drawing from multiple authoritative sources including "
            content += f"{', '.join([a['title'] for a in articles_list[:2]])}.\n"
            
        # Overview
        elif section_title in ["Overview", "Definition and Overview"]:
            content = f"## {section_title}\n\n"
            content += f"{topic} encompasses several key aspects:\n\n"
            for i, entity in enumerate(list(research['key_entities'].keys())[:5], 1):
                content += f"- **{entity}**: A fundamental component\n"
            content += f"\nThese elements work together to form the foundation of {topic}.\n"
            
        # History
        elif section_title in ["History", "History and Development"]:
            content = f"## {section_title}\n\n"
            content += f"The development of {topic} has evolved significantly over time. "
            content += f"Key milestones include foundational research and practical applications "
            content += f"that have shaped current understanding.\n"
            
        # How it Works / Principles
        elif section_title in ["How It Works", "Key Principles", "Key Concepts", "Theoretical Background"]:
            content = f"## {section_title}\n\n"
            content += f"The underlying mechanisms of {topic} involve:\n\n"
            entities = list(research['key_entities'].keys())[:6]
            for i, entity in enumerate(entities, 1):
                content += f"{i}. **{entity}**: Core principle\n"
            content += f"\nThese principles interact to produce the observed phenomena.\n"
            
        # Applications
        elif section_title == "Applications":
            content = f"## {section_title}\n\n"
            content += f"{topic} has found numerous practical applications:\n\n"
            categories = list(research['categories'].keys())[:4]
            for cat in categories:
                content += f"- **{cat}**: Practical implementations\n"
            content += f"\nThese applications demonstrate the versatility of {topic}.\n"
            
        # Challenges
        elif section_title in ["Challenges and Limitations", "Advantages and Disadvantages"]:
            content = f"## {section_title}\n\n"
            content += f"Despite its benefits, {topic} faces several challenges:\n\n"
            content += f"- Complexity and scalability\n"
            content += f"- Resource requirements\n"
            content += f"- Practical constraints\n"
            content += f"\nOngoing research aims to address these limitations.\n"
            
        # Future Directions / Current Research
        elif section_title in ["Future Directions", "Current Research"]:
            content = f"## {section_title}\n\n"
            content += f"Research in {topic} continues to evolve, with emerging areas including:\n\n"
            content += f"- Advanced methodologies\n"
            content += f"- Novel applications\n"
            content += f"- Theoretical improvements\n"
            content += f"\nThese directions promise to expand the impact of {topic}.\n"
            
        else:
            # Generic section
            content = f"## {section_title}\n\n"
            content += f"This section covers important aspects of {topic}. "
            content += f"Further details can be found in the referenced sources.\n"
        
        return content

# Initialize writing agent
writing_agent = WritingAgent(articles)
print("✓ Writing Agent initialized")

## Step 8: Implement Verification Agent

Agent that adds citations and verification metadata.

In [ ]:
class VerificationAgent:
    """Agent responsible for fact-checking and citations"""
    
    def __init__(self):
        self.name = "Verification Agent"
    
    def verify(self, state: ArticleState) -> ArticleState:
        """Add citations and verification metadata"""
        print(f"\n{self.name}: Adding citations and verification...")
        
        research = state.research_data
        articles_used = research['relevant_articles']
        
        # Create citations from sources
        citations = []
        for i, article_info in enumerate(articles_used, 1):
            citation = {
                'id': i,
                'title': article_info['title'],
                'url': article_info['url'],
                'type': 'wikipedia',
                'accessed': datetime.now().strftime('%Y-%m-%d')
            }
            citations.append(citation)
        
        # Verification summary
        verification_results = {
            'total_sources': len(articles_used),
            'citations_added': len(citations),
            'verification_method': 'source_based',
            'confidence_score': 0.85,  # Simplified
            'status': 'verified'
        }
        
        state.citations = citations
        state.verification_results = verification_results
        state.add_message(self.name, 
                         f"Added {len(citations)} citations",
                         citations=len(citations))
        
        print(f"  ✓ Added {len(citations)} citations")
        print(f"  ✓ Confidence score: {verification_results['confidence_score']:.2f}")
        
        return state

# Initialize verification agent
verification_agent = VerificationAgent()
print("✓ Verification Agent initialized")

## Step 9: Implement Article Assembly Agent

Agent that assembles the final article.

In [ ]:
class AssemblyAgent:
    """Agent responsible for assembling the final article"""
    
    def __init__(self):
        self.name = "Assembly Agent"
    
    def assemble(self, state: ArticleState) -> ArticleState:
        """Assemble all sections into final article"""
        print(f"\n{self.name}: Assembling final article...")
        
        article_parts = []
        
        # Title
        article_parts.append(f"# {state.topic}\n")
        article_parts.append(f"*Generated by Agentic AI Wikipedia Generator*\n")
        article_parts.append(f"*Date: {datetime.now().strftime('%Y-%m-%d')}*\n\n")
        article_parts.append("---\n\n")
        
        # Sections
        for section_title in state.outline:
            if section_title in state.sections:
                article_parts.append(state.sections[section_title])
                article_parts.append("\n")
        
        # See Also
        article_parts.append("## See Also\n\n")
        for i, article in enumerate(state.research_data['relevant_articles'][:5], 1):
            article_parts.append(f"- [{article['title']}]({article['url']})\n")
        article_parts.append("\n")
        
        # References
        article_parts.append("## References\n\n")
        for citation in state.citations:
            article_parts.append(
                f"[{citation['id']}] {citation['title']}. "
                f"Wikipedia. Retrieved {citation['accessed']}. "
                f"{citation['url']}\n\n"
            )
        
        # Metadata footer
        article_parts.append("\n---\n\n")
        article_parts.append("### Generation Metadata\n\n")
        article_parts.append(f"- **Sources Used**: {len(state.citations)}\n")
        article_parts.append(f"- **Sections**: {len(state.sections)}\n")
        article_parts.append(f"- **Key Entities**: {len(state.research_data['key_entities'])}\n")
        article_parts.append(f"- **Confidence Score**: {state.verification_results['confidence_score']:.2f}\n")
        article_parts.append(f"- **Status**: {state.verification_results['status']}\n")
        
        final_article = "".join(article_parts)
        state.final_article = final_article
        state.add_message(self.name, 
                         f"Assembled article with {len(final_article)} characters",
                         length=len(final_article))
        
        print(f"  ✓ Article assembled: {len(final_article)} characters")
        print(f"  ✓ Includes {len(state.sections)} content sections")
        print(f"  ✓ With {len(state.citations)} citations")
        
        return state

# Initialize assembly agent
assembly_agent = AssemblyAgent()
print("✓ Assembly Agent initialized")

## Step 10: Create Agent Orchestrator

Orchestrator that coordinates the workflow.

In [ ]:
class ArticleOrchestrator:
    """Orchestrates the multi-agent workflow"""
    
    def __init__(self, research_agent, planning_agent, writing_agent, 
                 verification_agent, assembly_agent):
        self.research_agent = research_agent
        self.planning_agent = planning_agent
        self.writing_agent = writing_agent
        self.verification_agent = verification_agent
        self.assembly_agent = assembly_agent
    
    def generate_article(self, topic: str) -> ArticleState:
        """Execute the complete article generation pipeline"""
        print(f"\n{'='*60}")
        print(f"ARTICLE GENERATION PIPELINE")
        print(f"Topic: {topic}")
        print(f"{'='*60}")
        
        # Initialize state
        state = ArticleState(topic=topic)
        
        # Execute agents in sequence
        state = self.research_agent.research(state)
        state = self.planning_agent.plan(state)
        state = self.writing_agent.write(state)
        state = self.verification_agent.verify(state)
        state = self.assembly_agent.assemble(state)
        
        print(f"\n{'='*60}")
        print(f"PIPELINE COMPLETE!")
        print(f"{'='*60}")
        print(f"Generated article: {len(state.final_article)} characters")
        print(f"Total agents executed: 5")
        print(f"Messages exchanged: {len(state.messages)}")
        
        return state

# Initialize orchestrator
orchestrator = ArticleOrchestrator(
    research_agent=research_agent,
    planning_agent=planning_agent,
    writing_agent=writing_agent,
    verification_agent=verification_agent,
    assembly_agent=assembly_agent
)

print("✓ Orchestrator initialized and ready!")

## Step 11: Generate Test Article

Let's generate our first Wikipedia-style article!

In [ ]:
# Generate an article
test_topic = "Deep Learning"

result_state = orchestrator.generate_article(test_topic)

# Display a preview
print(f"\n{'='*60}")
print("ARTICLE PREVIEW (First 1000 characters)")
print(f"{'='*60}\n")
print(result_state.final_article[:1000] + "...")

## Step 12: Display Complete Article

View the complete generated article.

In [ ]:
# Display the full article
from IPython.display import Markdown, display

display(Markdown(result_state.final_article))

## Step 13: Save Generated Article

Save the article to a file.

In [ ]:
# Save article to file
article_filename = OUTPUTS_DIR / f"{test_topic.replace(' ', '_')}_generated.md"
with open(article_filename, 'w', encoding='utf-8') as f:
    f.write(result_state.final_article)

print(f"✓ Article saved to: {article_filename}")
print(f"  File size: {article_filename.stat().st_size / 1024:.2f} KB")

## Step 14: Generate Multiple Articles

Generate articles for different topics.

In [ ]:
# Generate articles for multiple topics
test_topics = [
    "Machine Learning",
    "Quantum Computing",
    "Natural Language Processing"
]

generated_articles = {}

for topic in test_topics:
    print(f"\nGenerating article for: {topic}")
    print("-" * 60)
    
    state = orchestrator.generate_article(topic)
    generated_articles[topic] = state
    
    # Save article
    filename = OUTPUTS_DIR / f"{topic.replace(' ', '_')}_generated.md"
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(state.final_article)
    
    print(f"✓ Saved: {filename}")

print(f"\n{'='*60}")
print(f"Generation Complete!")
print(f"{'='*60}")
print(f"Total articles generated: {len(generated_articles)}")
print(f"Articles saved to: {OUTPUTS_DIR}")

## Step 15: Analyze Generation Statistics

Analyze the article generation process.

In [ ]:
# Create analysis summary
analysis_data = []

for topic, state in generated_articles.items():
    analysis_data.append({
        'Topic': topic,
        'Length (chars)': len(state.final_article),
        'Sections': len(state.sections),
        'Citations': len(state.citations),
        'Sources': state.research_data['total_sources'],
        'Key Entities': len(state.research_data['key_entities']),
        'Confidence': state.verification_results['confidence_score'],
        'Agent Messages': len(state.messages)
    })

analysis_df = pd.DataFrame(analysis_data)

print("Article Generation Statistics:")
print("=" * 80)
print(analysis_df.to_string(index=False))
print("\nSummary:")
print(f"  Average article length: {analysis_df['Length (chars)'].mean():.0f} characters")
print(f"  Average sections per article: {analysis_df['Sections'].mean():.1f}")
print(f"  Average citations per article: {analysis_df['Citations'].mean():.1f}")
print(f"  Average sources used: {analysis_df['Sources'].mean():.1f}")

## Step 16: Project Summary

Final summary of the complete system.

In [ ]:
print(f"\n{'='*80}")
print("PROJECT COMPLETE: Agentic AI-Powered Wikipedia Article Generator")
print(f"{'='*80}\n")

print("✅ Phase 1: Data Collection & Preprocessing")
print(f"   - Collected 1,337 Wikipedia articles")
print(f"   - Extracted entities and relationships")
print(f"   - Built knowledge graph with 11,091 edges")
print(f"   - Generated 384-dim embeddings")

print("\n✅ Phase 2: GraphRAG Engine")
print(f"   - Built FAISS vector index")
print(f"   - Implemented graph traversal")
print(f"   - Created hybrid retrieval system")
print(f"   - Fusion ranking algorithm")

print("\n✅ Phase 3: Multi-Agent System")
print(f"   - Research Agent: Information gathering")
print(f"   - Planning Agent: Article structuring")
print(f"   - Writing Agent: Content generation")
print(f"   - Verification Agent: Citations & validation")
print(f"   - Assembly Agent: Final compilation")
print(f"   - Orchestrator: Workflow coordination")

print(f"\n{'='*80}")
print("System Capabilities:")
print(f"{'='*80}")
print("✓ Automatic article generation from topics")
print("✓ Multi-source information retrieval")
print("✓ Graph-based + semantic search")
print("✓ Structured content with citations")
print("✓ Verification metadata")
print("✓ Scalable agent architecture")

print(f"\n{'='*80}")
print("Generated Articles:")
print(f"{'='*80}")
for topic in generated_articles.keys():
    filename = OUTPUTS_DIR / f"{topic.replace(' ', '_')}_generated.md"
    print(f"  • {topic}: {filename}")

print(f"\n{'='*80}")
print("🎉 All phases complete!")
print(f"{'='*80}")